In [28]:
from hydra import compose, initialize_config_module
from omegaconf import DictConfig
import monai
import torch
import hydra
from dataclasses import dataclass, field

In [29]:
hparams_dict = {
    "finetune_frac": 0.1,
    "test_frac": 0.1,
    "train_batchsize": 1024,
    "ftune_batchsize": 1024,
    "num_workers": 2,
    "device": "cpu",
    "spatial_dims": 2,
    "in_channels": 1,
    "out_channels": 5,
    "loss": "CrossEntropyLoss",
    "optimizer": "AdamW",
    "epochs": 4,
    "val_interval": 1,
    "lr": 1e-5,
}


In [42]:
from hydra.core.config_store import ConfigStore

@dataclass
class OptimizerConfig:
    _target_: str = "torch.optim.AdamW"
    lr: float = 0.001

@dataclass
class ModelConfig:
    _target_: str = "monai.networks.nets.DenseNet121"
    spatial_dims: int = 3
    in_channels: int = 1
    out_channels: int = 2

@dataclass
class OptimizerConfig:
    _target_: str = "torch.optim.AdamW"
    lr: float = 0.001

@dataclass
class LossFunctionConfig:
    _target_: str = "torch.nn.CrossEntropyLoss"

@dataclass
class Config:
    model: ModelConfig = field(default_factory=ModelConfig)
    optimizer: OptimizerConfig = field(default_factory=OptimizerConfig)
    loss_fn: LossFunctionConfig = field(default_factory=LossFunctionConfig)


cs = ConfigStore.instance()
cs.store(name="config", node=Config)

In [52]:
from typing import Any
def instantiate_class(class_path: str, *args, **kwargs) -> Any:
    module_path, class_name = class_path.rsplit('.', 1)
    module = __import__(module_path, fromlist=[class_name])
    class_ = getattr(module, class_name)
    return class_(*args, **kwargs)

In [57]:
@dataclass
class TrainingConfig:
    batch_size: int = 64
    learning_rate: float = 0.001
    max_epochs: int = 10
    loss_function: str = "torch.nn.CrossEntropyLoss"
    optimizer: str = 'torch.optim.AdamW'

In [58]:
cfg_training = TrainingConfig()

In [60]:
instantiate_class(cfg_training)

AttributeError: 'TrainingConfig' object has no attribute 'rsplit'

In [53]:
@hydra.main(version_base=None, config_name="config")
def build_pipeline(cfg: Config):
    # Instantiate model, optimizer, and loss function
    model = hydra.utils.instantiate(cfg)
    optimizer = hydra.utils.instantiate(cfg.optimizer, params=model.parameters())
    loss_fn = hydra.utils.instantiate(cfg.loss_fn)
    return None

build_pipeline()


Error executing job with overrides: []
Error in call to target 'torch.optim.adamw.AdamW':
TypeError("AdamW.__init__() missing 1 required positional argument: 'params'")
full_key: optimizer

Set the environment variable HYDRA_FULL_ERROR=1 for a complete stack trace.


SystemExit: 1

/Users/vigneshkannan/Documents/Projects/mednist_trials/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [45]:
cfg.model

NameError: name 'cfg' is not defined

In [12]:
@hydra.main(
        config_path="../data/", config_name="hyperparam_mps", version_base=None)
def get_config(cfg: DictConfig) -> DictConfig:
    return cfg

In [13]:
cfg = get_config()

usage: ipykernel_launcher.py [--help] [--hydra-help] [--version]
                             [--cfg {job,hydra,all}] [--resolve]
                             [--package PACKAGE] [--run] [--multirun]
                             [--shell-completion] [--config-path CONFIG_PATH]
                             [--config-name CONFIG_NAME]
                             [--config-dir CONFIG_DIR]
                             [--experimental-rerun EXPERIMENTAL_RERUN]
                             [--info [{all,config,defaults,defaults-tree,plugins,searchpath}]]
                             [overrides ...]
ipykernel_launcher.py: error: unrecognized arguments: --f=/Users/vigneshkannan/Library/Jupyter/runtime/kernel-v2-892jed9sQxFzcX8.json


SystemExit: 2

In [26]:
import sys
sys.argv = sys.argv[:1]

config_path="../data"
config_name = "hyperparam_mps.yml"
with hydra.initialize(version_base=None, config_path=config_path):
    cfg = compose(config_name=config_name)
cfg = hydra.compose(config_name=config_name,)

MissingConfigException: Cannot find primary config 'hyperparam_mps.yml'. Check that it's in your config search path.

Config search path:
	provider=hydra, path=pkg://hydra.conf
	provider=main, path=file:///Users/vigneshkannan/Documents/Projects/mednist_trials/data
	provider=schema, path=structured://